# ADPA Performance Optimization

This notebook demonstrates advanced performance optimization techniques in ADPA, including:
- Query optimization
- Caching strategies
- Batch processing
- Performance monitoring

## Setup

First, let's import the necessary components and set up our environment.

In [ ]:
from adpa import Text2SQL, Database
from adpa.config import Config
from adpa.monitoring import PerformanceMonitor
from adpa.optimization import QueryOptimizer, QueryCache
from adpa.utils.profiling import profile_query

# Initialize components
config = Config()
config.load_env()

db = Database(config)
text2sql = Text2SQL(config)
optimizer = QueryOptimizer(config)
cache = QueryCache(config)
monitor = PerformanceMonitor(config)

## Query Optimization

Demonstrate how ADPA optimizes complex queries for better performance.

In [ ]:
# Complex query example
query = """
Show me the average salary by department,
including only departments with more than 5 employees,
and list the top 3 highest paid employees in each department,
sorted by department size
"""

# Profile the original query
with profile_query() as profiler:
    # Get the initial SQL
    sql = text2sql.translate(query)
    print(f"\nOriginal SQL:\n{sql}")
    
    # Get execution plan
    plan = optimizer.explain_plan(sql)
    print(f"\nOriginal Execution Plan:\n{plan}")
    
    # Get optimization suggestions
    suggestions = optimizer.analyze(sql)
    print(f"\nOptimization Suggestions:\n{suggestions}")
    
    # Optimize the query
    optimized_sql = optimizer.optimize(sql)
    print(f"\nOptimized SQL:\n{optimized_sql}")
    
    # Compare execution plans
    optimized_plan = optimizer.explain_plan(optimized_sql)
    print(f"\nOptimized Execution Plan:\n{optimized_plan}")

# Show profiling results
print("\nProfiling Results:")
print(profiler.summary())

## Caching Strategies

Show how ADPA uses caching to improve performance.

In [ ]:
# Configure cache settings
cache.configure(
    max_size="1GB",
    ttl=3600,  # 1 hour
    strategy="lru"
)

# Example queries for cache demonstration
queries = [
    "Show total employees by department",
    "List departments with average salary above 70000",
    "Show total employees by department"  # Repeated query
]

for query in queries:
    print(f"\nExecuting query: {query}")
    
    # Check cache
    cache_key = cache.generate_key(query)
    cached_result = cache.get(cache_key)
    
    if cached_result:
        print("Cache hit! Using cached result")
        result = cached_result
    else:
        print("Cache miss. Executing query...")
        # Execute and cache
        with monitor.measure_execution_time():
            result = text2sql.execute(query)
            cache.set(cache_key, result)
    
    print(f"Results: {result}")

# Show cache statistics
print("\nCache Statistics:")
print(cache.get_stats())

## Batch Processing

Demonstrate efficient batch processing techniques.

In [ ]:
# Example batch queries
batch_queries = [
    "Update department budget for Engineering",
    "Update department budget for Marketing",
    "Update department budget for Sales",
    "Update employee count for Engineering",
    "Update employee count for Marketing",
    "Update employee count for Sales"
]

# Compare individual vs batch execution
print("Individual Execution:")
with monitor.measure_execution_time() as individual_timer:
    for query in batch_queries:
        text2sql.execute(query)
print(f"Individual execution time: {individual_timer.elapsed_time}")

print("\nBatch Execution:")
with monitor.measure_execution_time() as batch_timer:
    text2sql.execute_batch(batch_queries)
print(f"Batch execution time: {batch_timer.elapsed_time}")

# Show performance comparison
speedup = individual_timer.elapsed_time / batch_timer.elapsed_time
print(f"\nSpeedup factor: {speedup:.2f}x")

## Performance Monitoring

Show ADPA's performance monitoring capabilities.

In [ ]:
# Enable detailed performance monitoring
monitor.set_level('DEBUG')

# Run a performance test suite
with monitor.session("performance_test") as session:
    # Test query translation performance
    with session.measure("translation"):
        for _ in range(10):
            text2sql.translate("Show me all employees")
    
    # Test query execution performance
    with session.measure("execution"):
        for _ in range(10):
            text2sql.execute("SELECT * FROM employees")
    
    # Test cache performance
    with session.measure("caching"):
        for _ in range(10):
            cache_key = cache.generate_key("SELECT * FROM employees")
            cache.get(cache_key)

# Generate performance report
report = monitor.generate_report()
print("\nPerformance Report:")
print(report)

# Show performance metrics
print("\nPerformance Metrics:")
metrics = monitor.get_metrics()
for metric in metrics:
    print(f"{metric.name}: {metric.value} {metric.unit}")

## Cleanup

Clean up resources and save performance data.

In [ ]:
# Save performance data
monitor.save_metrics("performance_metrics.json")

# Clear cache
cache.clear()

# Shutdown monitoring
monitor.shutdown()

# Close database connection
db.close()